In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
import sys
sys.path.append('/Users/dbao/Downloads/research/rc_scripts/analysis')

from final_analysis.process_helper.df_choice_add_features import df_choice_add_features
from final_analysis.process_helper.df_undo_start_end import df_undo_start_end
from final_analysis.process_helper.df_undo_seq import df_undo_seq

# Load data

In [3]:
home_dir = '/Users/dbao/Library/CloudStorage/GoogleDrive-sherrydbao@gmail.com/.shortcut-targets-by-id/1sYZNVBbYCrHcHjo-9xdKcA4wLnob9ZEq/'+'/road_construction/data/2022_online/'
data_dir = home_dir + 'R_analysis_data/'
out_dir = home_dir + 'figures/figures_final/'
out_dir2 = home_dir + 'stat_model/final/'

In [4]:
data_choice_level = pd.read_csv(data_dir +  'choice_level/choicelevel_data.csv') # first column is index  
data_choice_level = data_choice_level.rename(columns={'Unnamed: 0':'index_copy'})
data_choice_level = df_choice_add_features(data_choice_level)
sc_data_choice_level = data_choice_level[data_choice_level['condition']==1].reset_index(drop=True) 

## process for undo sequence

In [5]:
condition = f"(lastUndo == 1)"
accu_error = df_undo_start_end(sc_data_choice_level, condition)

In [6]:
accu_error

,subjects,puzzleID,cum_error_beginning,cum_error_end,n_connect_beginning,n_connect_end,undo_length,mas_gain,error_beginning,error_count_beginning,sequential_single,terminal_beginning,category
13,0,0,1,0,7,1,6,1,0,9,False,1,0
11,0,1,0,0,6,1,5,0,0,9,False,1,0
17,0,1,2,0,4,1,3,2,0,11,False,1,1
24,0,1,1,0,5,2,3,1,0,10,False,1,1
16,0,2,0,0,9,2,7,0,0,72,False,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,99,38,1,0,7,6,1,1,1,-673,True,1,1
4,99,40,0,0,2,0,2,0,0,-209,False,0,0
8,99,42,1,0,7,6,1,1,1,-58,True,1,1
10,99,44,1,0,5,0,5,1,0,-117,False,1,1


# get test statistic

In [7]:
avg_sub_masgain = accu_error.groupby(['subjects'])['mas_gain'].mean().mean()

std_sub_masgain = accu_error.groupby(['subjects'])['mas_gain'].mean().std()
# get standard error
print(std_sub_masgain/np.sqrt(len(accu_error)))
print(avg_sub_masgain)

0.011006617340032387
1.018126284220514


# get undo length distribution for each subject

In [10]:
undo_length_map = accu_error.groupby(['subjects','n_connect_beginning'])['undo_length'].apply(list).reset_index()
undo_length_map[undo_length_map.subjects == 2]

,subjects,n_connect_beginning,undo_length
15,2,1,"[1, 1]"
16,2,2,"[2, 1, 2]"
17,2,3,[3]
18,2,4,"[1, 3, 3, 4]"
19,2,5,"[5, 4, 4, 5, 4, 5, 1, 4, 1, 5]"
20,2,6,"[6, 4, 2, 4, 1, 2, 5, 1, 1, 4, 6, 3, 3, 6, 5, ..."
21,2,7,"[2, 1, 5, 6, 5, 3, 3, 7, 3, 2, 3, 4, 6, 4, 2, ..."
22,2,8,"[8, 8, 3, 8, 3, 7, 6, 8, 8, 8, 4, 5, 8, 6, 7, ..."
23,2,9,"[9, 5, 8, 3, 9, 9]"
24,2,10,"[10, 9]"


# get df for undo sequence

In [11]:
condition = f"(firstUndo == 1)"
undo_seqs = df_undo_seq(sc_data_choice_level, condition)

In [12]:
undo_seqs[undo_seqs['subjects']==2]

,subjects,puzzleID,n_connect_beginning,path_list,currMAS_list
0,2,0,8.0,"[0, 5, 26, 4, 24, 15, 7, 8, 3]","[8, 8, 8, 8, 8, 8, 8, 8, 8]"
1,2,0,7.0,"[0, 28, 12, 26, 4, 24, 15, 9]","[8, 8, 8, 8, 8, 8, 8, 7]"
2,2,0,7.0,"[0, 28, 12, 26, 4, 24, 7, 8]","[8, 8, 8, 8, 8, 8, 7, 8]"
3,2,0,7.0,"[0, 28, 12, 26, 4, 24, 7, 3]","[8, 8, 8, 8, 8, 8, 7, 7]"
4,2,0,7.0,"[0, 28, 12, 17, 26, 4, 24, 15]","[8, 8, 8, 7, 7, 8, 8, 8]"
...,...,...,...,...,...
1,2,42,6.0,"[0, 23, 17, 11, 24, 15, 6]","[8, 6, 6, 6, 6, 6, 6]"
2,2,42,7.0,"[0, 25, 5, 4, 22, 8, 1, 7]","[8, 8, 7, 8, 8, 8, 8, 8]"
0,2,44,5.0,"[0, 8, 20, 9, 4, 24]","[6, 6, 6, 6, 6, 5]"
1,2,44,5.0,"[0, 8, 20, 9, 4, 7]","[6, 6, 6, 6, 6, 6]"


# simulate for null distribution

In [13]:
# for each row of error_map, randomly sample 1 undo_length from undo_length_map
def get_sim_avg_mas_gain(undo_seqs, undo_length_map):
    results = []
    for i in range(len(undo_seqs)):
        subset = undo_seqs.iloc[i]
        n_connect_beginning = subset['n_connect_beginning']
        subjects = subset['subjects']

        filterd = undo_length_map[(undo_length_map['subjects']==subjects) & (undo_length_map['n_connect_beginning']==n_connect_beginning)]
        sampled_undo_length = random.choice(filterd['undo_length'].values[0])
    
        currMAS_list = subset['currMAS_list']
        sampled_mas_gain = currMAS_list[-sampled_undo_length-1] - currMAS_list[-1]
        results.append({'subjects': subjects, 'n_connect_beginning': n_connect_beginning, 'mas_gain': sampled_mas_gain})
    sampled_mas_gains = pd.DataFrame(results)
    avg = sampled_mas_gains.groupby(['subjects'])['mas_gain'].mean().mean()
    return avg

In [14]:
get_sim_avg_mas_gain(undo_seqs, undo_length_map)

0.8454062272049814

In [15]:
sim_list = []
for i in range(1000):
    sim_list.append(get_sim_avg_mas_gain(undo_seqs, undo_length_map))

In [20]:
# get the position of avg_sub_masgain in sim_list's quantile
sorted_list = sorted(sim_list)
# Get the position of the target in the sorted list
position = len([x for x in sorted_list if x < avg_sub_masgain])


print(avg_sub_masgain, np.mean(sorted_list))
# standard error
print(np.std(sorted_list)/np.sqrt(len(sorted_list)))
position/len(sorted_list)


1.018126284220514 0.842987609283836
0.00026457478756055844


1.0